In [2]:
# Imports
import sys
sys.path.insert(1, '/users/wx21978/projects/pion-phys/pi0-analysis/analysis/')
import os
import operator
import warnings
import numpy as np
import awkward as ak
import pandas as pd
import copy
from particle import Particle
import matplotlib.pyplot as plt
from python.analysis import EventSelection, Plots, vector, PairSelection, Master, PFOSelection, Tags
from apps import photon_pairs
import time

In [3]:
plt_conf = Plots.PlotConfig()
plt_conf.SHOW_PLOT = True
plt_conf.SAVE_FOLDER = None
# plt_conf.BINS = 30

In [16]:
_a_ = 1
_b_ = 2
x = 4

eval("(x>_a_)|(x<_b_)")

True

In [13]:
ord('a')

97

In [24]:
chr(97)

'a'

In [15]:
eval("np." + repr(np.array([1,2])))

array([1, 2])

In [1]:
def dumb_sum(num):
    def next_val(index):
        if index >= num:
            return index
        else:
            return index + next_val(index+1)
    return next_val(0)

dumb_sum(4)

10

In [116]:
def insert_values_to_func_str(func_str, values):
    for i in range(len(values)):
        func_str = func_str.replace(f"_{i}_", f"{values[i]}")
    return func_str

def cuts_to_func(values, *operations, func_str=None):
    ops = {
        "==": operator.eq,
        "!=": operator.ne,
        "<": operator.lt,
        "<=": operator.le,
        ">": operator.gt,
        ">=": operator.ge}

    if func_str is None:
        if len(operations) != 1:
            raise ValueError("operations must be sepcified if func_str is not specified")
        else:
            operations = operations[0]
        def cut_func(property_to_cut):
            def next_cut(index):
                curr_cut = ops[operations[index]](property_to_cut, values[index])
                if index <= 0:
                    return curr_cut
                else:
                    return np.logical_and(curr_cut, next_cut(index-1))
            return next_cut(len(values)-1)
        return cut_func
    else:
        formatted_func = insert_values_to_func_str(func_str, values)
        return lambda x: eval(formatted_func)

def cuts_to_str(values, *operations, func_str=None, name_format=False):
    str_ini = ", " if name_format else ""
    if func_str is None:
        if len(operations) != 1:
            raise ValueError("operations must be sepcified if func_str is not specified")
        else:
            operations = operations[0]
        
        if len(values) == 1:
            str_ini = "" if name_format else "x"
            return str_ini + f" {operations[0]} {values[0]}"
        elif len(values) == 2:
            if (">" in operations[0]) and ("<" in operations[1]):
                return str_ini + f"{values[0]} {operations[0].replace('>','<')} x {operations[1]} {values[1]}"
            elif (">" in operations[1]) and ("<" in operations[0]):
                return str_ini + f"{values[1]} {operations[1].replace('>','<')} x {operations[0]} {values[0]}"
        return str_ini + " and ".join([f"(x {op} {val})" for val, op in zip(values, operations)])

    else:
        return str_ini + insert_values_to_func_str(func_str, values)

In [48]:
"x==_a_ | x>_b_".replace(f"_{chr(ord('a') + 0)}_", f"values[{0}]")

'x==values[0] | x>_b_'

In [119]:
print(cuts_to_func([2,5], func_str="(x==_0_) or (x>_1_)")(5))
print(cuts_to_str([2,5], func_str="(x==_0_) or (x>_1_)"))
print(cuts_to_str([2,5], func_str="(x==_0_) or (x>_1_)", name_format=True))

False
(x==2) or (x>5)
, (x==2) or (x>5)


In [117]:
print(cuts_to_func([2,5], [">=", "<"])(5))
print(cuts_to_str([2,5], [">=", "<"]))
print(cuts_to_str([2,5], [">=", "<"], name_format=True))

False
2 <= x < 5
, 2 <= x < 5


In [118]:
print(cuts_to_func([5,10], ["==", ">"])(1))
print(cuts_to_str([5,10], ["==", ">"]))
print(cuts_to_str([5,10], ["==", ">"], name_format=True))

False
(x == 5) and (x > 10)
, (x == 5) and (x > 10)


In [115]:
print(cuts_to_func([10], ["<="])(1))
print(cuts_to_str([10], ["<="]))
print(cuts_to_str([10], ["<="], name_format=True))

True
x <= 10
 <= 10


In [ ]:
def cut_to_func(cut):
    if isinstance(cut, tuple):
        if len(cut) == 1:
            return lambda count: count >= cut[0]
        elif len(cut) == 2:
            return lambda count: np.logical_and(
                count >= min(cut), count <= max(cut))
        else:
            raise ValueError(f"Cut tuple {cut} must contain 1 or 2 values.")
    if isinstance(cut, list):
        if len(cut) == 1:
            return lambda count: count > cut[0]
        elif len(cut) == 2:
            return lambda count: np.logical_and(
                count > min(cut), count < max(cut))
        else:
            raise ValueError(f"Cut list {cut} must contain 1 or 2 values.")
    elif cut is None:
        return lambda count: True
    else:
        return lambda count: count == cut
    
def cut_to_str(cut):
    if isinstance(cut, tuple):
        if len(cut) == 1:
            return f" >= {cut[0]}"
        elif len(cut) == 2:
            return f", {min(cut)} <= X <= {max(cut)}"
        else:
            raise ValueError(f"Cut tuple {cut} must contain 1 or 2 values.")
    if isinstance(cut, list):
        if len(cut) == 1:
            return f" > {cut[0]}"
        elif len(cut) == 2:
            return f", {min(cut)} < X < {max(cut)}"
        else:
            raise ValueError(f"Cut list {cut} must contain 1 or 2 values.")
    elif cut is None:
        return ", no cut"
    else:
        return f" == {cut}"

In [7]:
# This needs to change! Need it to give the cumulative mask since the first mask

class MaskIter():
    def __init__(self, start_sigs, masks):
        self.sigs = start_sigs
        self.masks = masks
        self.iter_max = len(masks)
        self.index = 0
        self.curr_mask = self.masks[self.index]

        if self.iter_max != len(self.sigs):
            raise ValueError(f"masks and sigs must have the same length: {self.iter_max} and {len(self.sigs)}")
        if self.index >= self.iter_max:
            raise IndexError(f"Start index {self.index} is out of bounds for masks length of {self.iter_max}")

        self.mask_inds = list(range(self.iter_max))
        self.simultaneous_masks = []
        simul_list = [False] + [self.check_simul_masks(prev, this) for prev, this in zip(self.sigs[:-1], self.sigs[1:])]
        self.next_simul = simul_list[1:] + [False]
        self.init_simul = []
        last_simul = 0
        for i, this_simul in enumerate(simul_list):
            if not this_simul:
                last_simul = i
            self.init_simul.append(last_simul)
        return
    
    def __iter__(self):
        self.index = 0
        self.curr_mask = self.masks[0]
        return self

    def check_simul_masks(self, sig1, sig2):
        return (sig1[0] == sig2[0]) and ((sig1[1] == sig2[-1]) or (sig1[1] == -1) or (sig2[1] == -1))

    def get_mask_func(self, mask, next_simul):
        if next_simul:
            def apply_mask_func(data):
                return mask, data
        else:
            def apply_mask_func(data):
                return mask, data[mask]
        return apply_mask_func

    def __next__(self):
        if self.index >= self.iter_max:
            raise StopIteration

        if self.init_simul[self.index] == self.index:
            self.curr_mask = self.masks[self.index]
        else:
            self.curr_mask = np.logical_and(self.curr_mask, self.masks[self.index])
        self.index += 1
        return self.get_mask_func(self.curr_mask, self.next_simul[self.index-1])
        
    def __getitem__(self, sli : slice):
        if (sli.step is not None) and (sli.step < 0):
            rev_slice = slice(None, None, -1)
            sli = slice(sli.stop + 1, sli.start + 1, abs(sli.step))
        else:
            rev_slice = slice(None, None, None)
        this_index = self.mask_inds[sli]
        init_masks = self.init_simul[sli]
        next_simuls = self.next_simul[sli]
        res = []
        last_end = init_masks[0]
        curr_mask = self.masks[last_end]
        for this_i, init_i, next_simul in zip(this_index, init_masks, next_simuls):
            if init_i > last_end:
                last_end = init_i
                curr_mask = self.masks[last_end]
            for new_mask in self.masks[last_end+1:this_i+1]:
                curr_mask = np.logical_and(curr_mask, new_mask)
            res.append(self.get_mask_func(curr_mask, next_simul))
        return res[rev_slice]

In [86]:
class CutHandler():
    _default_particles = [
        211, -211, 13, -13,
        11, -11, 22, 2212, 321]
    _event_table_cols = np.array([
            "Remaining events", "Percentage of total events remaining",
            "Relative percentage events"])
    _pfo_table_cols = np.array([
            "Remaining PFOs", "Percentage of total PFOs remaining",
            "Relative percentage of PFOs", "Average PFOs per event"])
    _particle_table_cols = [
            "remaining", "percentage remaining",
            "relative percentage remaining", "average per event"]

    # @staticmethod
    # def _init_doc(func):
    #     documentation = """
    #     Create new a `CutHandler` object to store applied masks
    #     and return a table of tracked particles when asked.

    #     Parameters
    #     ----------
    #     evts : Master.Data, optional
    #         Initial events the masks are being applied from.
    #         If not set here, it must be set for getting a
    #         table using `self.set_init_evts(evts)`. Default
    #         is None.
    #     particles_to_tag : list, optional
    #         List of pdg codes to be tracked by the tables.
    #         Default is
    #     """ + f"{CutHandler._default_particles}." + """
        
    #     Returns
    #     -------
    #     CutHandler
    #         New `CutHandler` instance.
    #     """
    #     func.__doc__ = documentation
    #     return func
    # @_init_doc
    def __init__(
            self,
            evts: Master.Data = None,
            particles_to_tag: list = _default_particles):
        """
        Create new a `CutHandler` object to store applied masks
        and return a table of tracked particles when asked.

        Parameters
        ----------
        evts : Master.Data, optional
            Initial events the masks are being applied from.
            If not set here, it must be set for getting a
            table using `self.set_init_evts(evts)`. Default
            is None.
        particles_to_tag : list, optional
            List of pdg codes to be tracked by the tables.
            Default is [211, -211, 13, -13,11, -11, 22, 2212, 321]
        
        Returns
        -------
        CutHandler
            New `CutHandler` instance.
        """
        self._masks = []
        self._signatures = []
        self._start_sig = ()
        self._end_sig = ()
        self.curr_mask_index = 0
        self.concat_index = 0
        self._concat_indicies = [0]
        self._pfos_init = False
        self._init_data = None
        self._particle_tags = None
        self._data_changed = True
        self.init_events_set = False
        if evts is not None:
            self.set_init_evts(evts)
        self._last_pfo_counts = None
        self._names=[]

        self._particles = particles_to_tag

        self._table_data = {}
        return

    def _gen_basic_counts(self):
        init_evt_count = ak.num(self._init_data, axis=0)
        init_pfo_count = ak.count(self._init_data)
        results = {
            "Name":["Initial data"],
            "Remaining events":[init_evt_count],
            "Percentage of total events remaining":[100.],
            "Relative percentage events":[100.],
            "Remaining PFOs":[init_pfo_count],
            "Percentage of total PFOs remaining":[100.],
            "Relative percentage of PFOs":[100.],
            "Average PFOs per event":[init_pfo_count/init_evt_count]}
        last_evt_count = init_evt_count
        last_pfo_count = init_pfo_count
        data = self._init_data
        for name, application_func in zip(self._names, self):
            this_mask, new_data = application_func(data)
            this_evt_count = ak.num(data[this_mask], axis=0)
            this_pfo_count = ak.count(data[this_mask])
            results["Name"].append(name)
            results["Remaining events"].append(this_evt_count)
            results["Percentage of total events remaining"].append(100.*this_evt_count/init_evt_count)
            results["Relative percentage events"].append(100.*this_evt_count/last_evt_count)
            results["Remaining PFOs"].append(this_pfo_count)
            results["Percentage of total PFOs remaining"].append(100.*this_pfo_count/init_pfo_count)
            results["Relative percentage of PFOs"].append(100.*this_pfo_count/last_pfo_count)
            results["Average PFOs per event"].append(this_pfo_count/this_evt_count)
            last_evt_count = this_evt_count
            last_pfo_count = this_pfo_count
            data = new_data
        return results

    def _gen_particle_counts(self, init_list):
        init_count = np.sum(init_list)
        results = {
            "remaining": [init_count],
            "percentage remaining": [100.],
            "relative percentage remaining": [100.],
            "average per event": [init_count/ak.num(init_list, axis=0)]}
        data = init_list
        for application_func in self:
            this_mask, new_data = application_func(data)
            particle_count = np.sum(data[this_mask])
            results["remaining"].append(particle_count)
            results["percentage remaining"].append(100. * particle_count/init_count)
            results["relative percentage remaining"].append(
                100. * particle_count/results["remaining"][-2])
            results["average per event"].append(particle_count/ak.num(data[this_mask], axis=0))
            data = new_data
        return results

    # def GeneratePi0Tags(self, evts : Master.Data):
    #     photons_mask = evts.trueParticlesBT.pdg == 22
    #     return Tags.GeneratePi0Tags(evts, photons_mask)

    def set_init_evts(self, evts : Master.Data):
        """
        Set the initial event counts to be cut using the masks.
        This is automatically performed if as `Master.Data`
        object is passed in initialisation.
        
        This must be performed prior to generating a table.

        Use `self.init_data_set` for a boolean indicating if
        the events have been set.

        N.B. if we want to to this for truth, we must change evts
        to be an acutal akward array (i.e.
        `evts.recoParticles.number` for reco and
        `evts.trueParticles.number` for truth). At the moment,
        we take a Master.Data object an pull the
        `evts.recoParticles.number` from it.

        Parameters
        ----------
        evts : Master.Data
            Initial events the masks are being applied from.
        """
        self._init_data = ak.ones_like(evts.recoParticles.number, dtype=bool)
        self._init_data_sig = self._get_mask_signature(self._init_data)
        self._particle_tags = Tags.GenerateTrueParticleTags(evts)
        self._data_changed = True
        self.init_events_set = True
        return

    def _get_mask_signature(self, mask, end=False):
        flat_array = isinstance(ak.count(mask, axis=0), int)
        if not end:
            pfo_level = -1 if flat_array else ak.count(mask)
            start_sig = (ak.num(mask, axis=0), pfo_level)
            return start_sig
        if flat_array:
            end_sig = (ak.sum(mask), -1)
        else:
            end_sig = (ak.num(mask, axis=0), ak.sum(mask))
        return end_sig
    
    def _validate_signature(self, signature, raise_exception=True):
        if self._start_sig == ():
            return
        simul_mask = (self._start_sig[0] == signature[0]) and ((self._start_sig[-1] == signature[-1]) or (self._start_sig[-1] == -1) or (signature[-1] == -1))
        good_mask = simul_mask or (self._end_sig[0] == signature[0] and ((self._end_sig[-1] == signature[-1]) or (self._end_sig[-1] == -1) or (signature[-1] == -1)))
        if (not good_mask) and raise_exception:
            raise ValueError(f"Mask signature ({signature[0]} events, {signature[1]} PFOs) does not match"
                             + f" the required signature of ({self._start_sig[0]} events, "
                             + f"{self._start_sig[1]} PFOs) for a simultaenous mask, or ({self._end_sig[0]}"
                             + f" events, {self._end_sig[1]} PFOs) for a consequtive mask.")
        return good_mask

    def add_mask(self, mask: ak.Array, name: str):
        """
        Add a mask to the instance.

        Parameters
        ----------
        mask : ak.Array
            Boolean mask of the cut applied.
        name : str
            Name indicating what the cut did.
        """
        this_sig = self._get_mask_signature(mask)
        self._validate_signature(this_sig)
        self._start_sig = this_sig
        self._end_sig = self._get_mask_signature(mask, end=True)
        self._masks.append(mask)
        self._signatures.append(this_sig)
        self._names.append(name)
        self.curr_mask_index += 1
        return

    def _mask_appliers(self):
        return MaskIter(self._signatures, self._masks)

    def __iter__(self):
        return self._mask_appliers()

    def copy(self):
        return copy.deepcopy(self)
    
    def apply_masks(
            self, data: ak.Array,
            return_table: bool = False,
            application_concat_index: int = 0,
            application_true_index: int = None):
        """
        Apply the masks in the instance to the data.

        Parameters
        ----------
        data : ak.Array
            Data to which the masks are applied. Must
            match the signature of the first mask.
        return_table : bool, optional
            If `True`, the both the filtered mask and
            corresponding table will be returned. Default
            is False.
        application_concat_index : int, optional
            Initial concatenation from which to apply masks.
            If data has already been filtered, use this to
            change from where the mask application begins.
            Current concatenation index can be seen using
            `self.concate_index`. Default is 0.
        application_true_index : int, optional
            Selects an arbitray mask as the starting point from
            which to begin applying masks. This can be used
            selections have not been grouped via concatenations.
            Current mask index can be seen using
            `self.curr_mask_index`. This will override
            `initial_concat_index` if set. Default is None.
        
        Returns
        -------
        ak.Array
            Filtered data.
        pd.DataFrame, optional
            Cuts table, only returned if `return_table` is
            True.
        """
        # TODO exception throwing
        if application_true_index is not None:
            initial_index = application_true_index
        else:
            initial_index = self._concat_indicies[application_concat_index]
        result = data
        for application_func in self._mask_appliers()[initial_index:]:
            _, result = application_func(result)
        if return_table:
            return result, self.get_table()
        else:
            return result
    
    # def get_filters_list(self, application_concat_index=0):
    #     new_data = data
    #     for application_func in self._mask_appliers()[self._concat_indicies[application_concat_index]:]:
    #         mask, _ = application_func(result)
        

    def _gen_table(self):
        if self._init_data is None:
            raise Exception("Initial data not added, supply events using set_init_events.")
        if not self._data_changed:
            return
        self._table_data.update(self._gen_basic_counts())
        for particle in self._particles:
            p_data = self._gen_particle_counts(self._particle_tags[f"${Particle.from_pdgid(particle).latex_name}$"].mask)
            self._table_data.update(dict((f"{particle} {key}", value) for (key, value) in p_data.items()))
        self._data_changed = False
        return

        # if not self._check_signature(self._get_mask_signature(self._masks[0]), self._init_data_sig):
        #     raise ValueError(f"Initial data signature {self._init_data_sig} does not match the first avaiable mask {self._get_mask_signature(self._masks[0])}")
    
    def _gen_particle_cols_array(self, pdg):
        return np.array(list(map(lambda c: f"{pdg} {c}", self._particle_table_cols)))
    def _gen_particle_cols_array_read(self, pdg, latex):
        if latex:
            return np.array(list(map(lambda c: f"${Particle.from_pdgid(pdg).latex_name}$ {c}", self._particle_table_cols)))
        else:
            return np.array(list(map(lambda c: f"{Particle.from_pdgid(pdg)} {c}", self._particle_table_cols)))

    def _new_init_events(self, col, index, init_name):
        if index == 0:
            result = self._table_data[col]
        else:
            result = self._table_data[col][index:]
            if "percentage" in col.lower():
                if "relative" not in col.lower():
                    init_percent = result[0]
                    result = list(map(lambda p: p/init_percent, result))
                else:
                    result[0] = 100.
        if col == "Name":
            result[0] = init_name
        return result

    def get_table(
            self, init_data_name: str = "Initial data",
            initial_concat_index: int = 0,
            initial_true_index: int = None,
            latex: bool = False,
            particles_list: list = _default_particles,
            events: bool = True, pfos: bool = True,
            counts: bool = True, percent_remain: bool = True,
            relative_percent: bool = True, ave_per_event: bool = True):
        """
        Generate the cuts table for the masks shown to the instance.

        The initial events must have been set prior to calling this,
        either in the initialisation, or the `set_init_events`
        method.

        Parameters
        ----------
        init_data_name : str, optional
            Name to be displayed for the pre-cuts row. Default is
            `"Initial data"`
        initial_concat_index : int, optional
            Initial concatenation from which to generate the table
            This can be used to i.e. removing initial event
            selection cuts if they are not of interest. Current
            concatenation index can be seen using
            `self.concate_index`. Default is 0.
        initial_true_index : int, optional
            Selects an arbitray mask as the starting point from
            which to generate the table. This can be used
            selections have not been grouped via concatenations.
            Current mask index can be seen using
            `self.curr_mask_index`. This will override
            `initial_concat_index` if set. Default is None.
        latex : bool, optional
            If `True`, result is a string which may be copied
            directly into LaTeX. Default is False.
        particles_list: list, optional
            List of pdg codes of particles to be included in the
            table. Default is [211, -211, 13, -13,11, -11, 22,
            2212, 321]
        events: bool, optional
            If `True`, columns indicating the number of events
            present are included. Default is True.
        pfos: bool, optional
            If `True`, columns indicating the total number of
            PFOs present are included. Default is True.
        counts: bool, optional
            If `True`, columns indicating the number of
            occurances for each tracked item are included.
            Default is True.
        percent_remain: bool, optional
            If `True`, columns indicating the percentage of the
            intial count for each tracked item are included.
            Default is True.
        relative_percent: bool, optional
            If `True`, columns indicating the relative percentage
            remaining compared to the previous row for each
            tracked item are included. Default is True.
        ave_per_event: bool, optional
            If `True`, columns indicating the mean number of
            occurances per event for each tracked item are
            included. Not included for events data. Default is
            True.
        
        Returns
        -------
        pd.DataFrame or str
            Table of results as a DataFrame if `latex` is False,
            or a string if `latex` is True.
        """
        self._gen_table()

        if initial_concat_index > self.concat_index:
            raise IndexError(
                f"Concatenation index {initial_concat_index} out of range, "
                + f"only {self.concat_index} concatenation(s) present.")
        cols_to_use = ["Name"]
        col_names = ["Name"]
        cols_to_keep = np.array([counts, percent_remain, relative_percent, ave_per_event])
        if events:
            cols = self._event_table_cols[cols_to_keep[:-1]].tolist()
            cols_to_use += cols
            col_names += cols
        if pfos:
            cols = self._pfo_table_cols[cols_to_keep].tolist()
            cols_to_use += cols
            col_names += cols
        for p in particles_list:
            cols_to_use += self._gen_particle_cols_array(p)[cols_to_keep].tolist()
            col_names += self._gen_particle_cols_array_read(p, latex)[cols_to_keep].tolist()
        
        if initial_true_index is not None:
            initial_index = initial_true_index
        else:
            initial_index = self._concat_indicies[initial_concat_index]
        data = {}
        for col, name in zip(cols_to_use, col_names):
            data[name] = self._new_init_events(
                col, initial_index, init_data_name)
        if latex:
            return pd.DataFrame(data).to_latex()
        else:
            return pd.DataFrame(data)

    def concatenate(self, other, return_copy=False):
        if not isinstance(other, CutHandler):
            raise NotImplementedError("CutHandler object can only be added to another CutHandler object")
        # Make sure the new set is consequtive
        if not self._validate_signature(other._signatures[0], raise_exception=False):
            raise ValueError(f"Combining CutHandler objects requires consequtively matching signatures: "
                             + f"{self._end_sig} and {other._signatures[0]} do not match.")
        if return_copy:
            result = self.copy()
        else:
            result = self
        result._masks = other._masks
        result._signatures += other._signatures[1:]
        result._start_sig = other._start_sig
        result._end_sig = other._end_sig
        result.concat_index += 1
        result._concat_indicies += [i + len(result._concat_indicies) for i in other._concat_indicies]
        if (result._init_data is None) and (other._init_data is not None):
            result._init_data = other._init_data
            result._init_data_sig = other._init_data_sig
            result._data_changed = True
            result.init_events_set = other.init_events_set
            if result._particles != other._particles:
                warnings.warn("Concatenated data has a different set of watched particles, re-running set_init_evts() is recommended.")
            result._particle_tags = other._particle_tags
        if return_copy:
            return result
        else:
            return

    def __add__(self, other):
        return self.concatenate(other, return_copy=True)


In [9]:
file = "/scratch/wx21978/pi0/root_files/1GeV_beam_v4_prelim/Prod4a_1GeV_BeamSim_00_prelim.root"

evts = Master.Data(file,
                         nTuple_type="PDSPAnalyser",
                         nEvents=-1,
                         start=-1)

In [7]:
evts.recoParticles.number

<Array [[0, 1, 2, 3, ... 671, 672, 673, 674]] type='23660 * var * int32'>

In [11]:
print(ak.count(evts.recoParticles.nHits > 80))
print(ak.num(evts.recoParticles.nHits > 80, axis=0))

12267594
23660


In [25]:
np.array(CutHandler._particle_table_cols).dtype

dtype('<U29')

In [90]:
dir(CutHandler.__init__.__code__)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'co_argcount',
 'co_cellvars',
 'co_code',
 'co_consts',
 'co_filename',
 'co_firstlineno',
 'co_flags',
 'co_freevars',
 'co_kwonlyargcount',
 'co_lines',
 'co_linetable',
 'co_lnotab',
 'co_name',
 'co_names',
 'co_nlocals',
 'co_posonlyargcount',
 'co_stacksize',
 'co_varnames',
 'replace']

In [83]:
CutHandler.__init__.__code__

__annotations__: {}
__builtins__: {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'), '__build_class__': <built-in function __build_class__>, '__import__': <built-in function __import__>, 'abs': <built-in function abs>, 'all': <built-in function all>, 'any': <built-in function any>, 'ascii': <built-in function ascii>, 'bin': <built-in function bin>, 'breakpoint': <built-in function breakpoint>, 'callable': <built-in function callable>, 'chr': <built-in function chr>, 'compile': <built-in function compile>, 'delattr': <built-in function delattr>, 'dir': <built-in function dir>, 'divmod': <built-in function divmod>, 'eval': <built-in function eval>, 'exec': <built-in function exec>, 'for

In [85]:
for prop in dir(CutHandler.__init__):
    print(f"{prop}: {eval('CutHandler.__init__.'+prop)}")

__annotations__: {}
__builtins__: {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'), '__build_class__': <built-in function __build_class__>, '__import__': <built-in function __import__>, 'abs': <built-in function abs>, 'all': <built-in function all>, 'any': <built-in function any>, 'ascii': <built-in function ascii>, 'bin': <built-in function bin>, 'breakpoint': <built-in function breakpoint>, 'callable': <built-in function callable>, 'chr': <built-in function chr>, 'compile': <built-in function compile>, 'delattr': <built-in function delattr>, 'dir': <built-in function dir>, 'divmod': <built-in function divmod>, 'eval': <built-in function eval>, 'exec': <built-in function exec>, 'for

In [53]:
cutter = CutHandler(evts)

In [54]:
cutter.add_mask(evts.recoParticles.nHits > 80, "nhits<80")
cutter.add_mask(np.logical_and(evts.recoParticles.energy > 50, evts.recoParticles.energy < 250), "50< p <250")

In [94]:
cutter.get_table(particles_list=[22, 211], ave_per_event=False, relative_percent=False, events=False, latex=True)

ImportError: Missing optional dependency 'Jinja2'. DataFrame.style requires jinja2. Use pip or conda to install Jinja2.

In [51]:
cutter.get_table(particles_list=[22, 211], ave_per_event=False, relative_percent=False, events=False)

,Name,Remaining PFOs,Percentage of total PFOs remaining,$gamma$ remaining,$gamma$ percentage remaining,$pi+$ remaining,$pi+$ percentage remaining
0,Initial data,12267594,100.000000,827505,100.000000,19662,100.000000
1,nhits<80,1680902,13.701970,20005,2.417508,13112,66.687010
2,50< p <250,426290,3.474928,10387,1.255219,7904,40.199369


In [79]:
pd.DataFrame(cutter._table_data)

,Name,Remaining events,Percentage of total events remaining,Relative percentage events,Remaining PFOs,Percentage of total PFOs remaining,Relative percentage of PFOs,Average PFOs per event,$\pi^{+}$ remaining,$\pi^{+}$ percentage remaining,...,$\gamma$ relative percentage remaining,$\gamma$ average per event,$p$ remaining,$p$ percentage remaining,$p$ relative percentage remaining,$p$ average per event,$K^{+}$ remaining,$K^{+}$ percentage remaining,$K^{+}$ relative percentage remaining,$K^{+}$ average per event
0,Initial data,23660,100.0,100.0,12267594,100.000000,100.000000,518.495097,19662,100.000000,...,100.000000,34.974852,83979,100.000000,100.000000,3.549408,36,100.000000,100.000000,0.001522
1,nhits<80,23660,100.0,100.0,1680902,13.701970,13.701970,71.044041,13112,66.687010,...,2.417508,0.845520,23616,28.121316,28.121316,0.998140,23,63.888889,63.888889,0.000972
2,50< p <250,23660,100.0,100.0,426290,3.474928,25.360788,18.017329,7904,40.199369,...,51.922019,0.439011,13987,16.655354,59.226795,0.591167,17,47.222222,73.913043,0.000719


In [ ]:
cutter._gen_table()
print(cutter._table_data)

In [34]:
class CutHandlerOld():
    def __init__(self):
        self._curr_masks = []
        self._conseq_masks = []
        self._signatures = []
        # self._signature = () # OLD
        self._start_sig = ()
        self._end_sig = ()
        # self._consequtive = () # OLD
        self.concat_index = 0
        self._concat_indicies = [0]
        self._pfos_init = False
        self._pre_pfo_init_masks = []
        self._init_pfo_count = np.inf
        self._last_pfo_counts = None
        self._pfo_table_cols = [
            "Name", "Remaining events", "Percentage of total events remaining",
            "Relative percentage events", "Remaining PFOs",
            "Percentage of total PFOs remaining", "Relative percentage of PFOs",
            "Average PFOs per event"]
        self._event_table_cols = [
            "Name", "Remaining events", "Percentage of total events remaining",
            "Relative percentage events"]
        self._event_inds_indicies = []

        self._table_data = {}
        for col in self._pfo_table_cols:
            self._table_data.update({col:[]})
    
    def __iter__(self):
        return MaskIter(self._signatures, self._masks, init_step=self.curr_concat_index)

    def copy(self):
        return copy.deepcopy(self)

    def _get_mask_signature(self, mask, end=False, update=False):
        flat_array = isinstance(ak.count(mask, axis=0), int)
        if update or (not end):
            pfo_level = -1 if flat_array else ak.count(mask)
            start_sig = (ak.num(mask, axis=0), pfo_level)
            if not update:
                return start_sig
        if flat_array:
            end_sig = (ak.sum(mask), -1)
        else:
            end_sig = (ak.num(mask, axis=0), ak.sum(mask))
        if not update:
            return end_sig
        # if update is True
        self._start_sig = start_sig
        self._end_sig = end_sig
        return
    
    def _get_last_pfo_sig(self):
        if not self._pfos_init:
            return -1, -1
        else:
            last_full = self._table_data["Percentage of total PFOs remaining"]/100
            last_relative = self._table_data["Relative percentage of PFOs"]/100
            last_start = round(last_full * self._init_pfo_count)
            last_end = round(last_start/last_relative)
            return last_start, last_end

    def _check_signature(self, signature):
        # simul_mask = (self._start_sig[0] == signature[0]) and ((start_pfo == signature[-1]) or (start_pfo == -1) or (signature[-1] == -1))
        # good_mask = simul_mask or (self._end_sig[0] == signature[0] and ((end_pfo == signature[-1]) or (end_pfo == -1) or (signature[-1] == -1)))
        good_mask = False
        if signature[-1] != -1:
            if self._start_sig[1] == -1:
                start_pfo, end_pfo = self._get_last_pfo_sig()
            else:
                start_pfo = self._start_sig[1]
                end_pfo = self._end_sig[1]
        else:
            start_pfo = -1
            end_pfo = -1
        if (self._start_sig[0] == signature[0]):
            simul_mask = True
            good_mask = (start_pfo == signature[-1]) or (start_pfo == -1)
        elif (self._end_sig[0] == signature[0]):
            simul_mask = False
            good_mask = (end_pfo == signature[-1]) or (end_pfo == -1)
        if not good_mask:
            raise ValueError(f"Mask signature ({signature[0]} events, {signature[1]} PFOs) does not match"
                             + f" the required signature of ({self._start_sig[0]} events, "
                             + f"{self._start_sig[1]} PFOs) for a simultaenous mask, or ({self._end_sig[0]}"
                             + f" events, {self._end_sig[1]} PFOs) for a consequtive mask.")
        return simul_mask

    # def _validate_signature(self, signature):
    #     if not self._check_signature(signature):
    #         if self._consequtive:
    #             conseq_text = "consequtive masks (each mask has input signature of the output of the previous mask)"
    #         else:
    #             conseq_text = "initially matched masks (each mask must have the same input signature)"
    #         raise ValueError(f"Mask signature ({signature[0]} events, {signature[1]} PFOs) does not match"
    #                             + f" the required signature for {conseq_text}.\nRequired signature is "
    #                             + f"({self._signature[0]} events, {self._signature[1]} PFOs).")

    def _add_initial_data(self, mask):
        self._signatures += [self._get_mask_signature(mask)]
        self._get_mask_signature(mask, update=True)
        table_data = [
            "Initial data",
            ak.num(mask, axis=0),
            100.,
            100.]
        if self._start_sig[1] == -1:
            table_data += [
                "Not supplied","-","-","-"]
        else:
            self._pfos_init = True
            self._init_pfo_count = ak.count(mask)
            self._last_pfo_counts = ak.count(mask, axis=1)
            table_data += [
                ak.count(mask),
                100.,
                100.,
                ak.count(mask)/ak.num(mask, axis=0)]
        self._append_table_data(table_data)
        return

    def _get_remaining_events_data(self):
        this_sig = self._signatures[-1]
        if this_sig[1] == -1:
            curr_remaining = ak.sum(self.get_curr_mask())
        else:
            curr_remaining = ak.num(self.get_curr_mask(), axis=0)
        init_remaining = self._table_data["Remaining events"][0]
        last_remaining = self._table_data["Remaining events"][-1]
        return [curr_remaining,
                100 * curr_remaining/init_remaining,
                100 * curr_remaining/last_remaining]
    
    def _get_remaining_pfos_data(self):
        this_sig = self._signatures[-1]
        if not self._pfos_init:
            if this_sig[1] == -1:
                self._pre_pfo_init_masks += [self.get_curr_mask()]
                return ["-", "-", "-", "-"]
            else:
                self._pfos_init = True
                this_mask = self.get_curr_mask()
                self._init_pfo_count = ak.count(this_mask)
                last_remaining = ak.sum(self._init_pfo_count)
        else:
            last_remaining = self._table_data["Remaining PFOs"][-1]
        if this_sig[1] == -1:
            this_mask = self.get_curr_mask()
            curr_remaining = ak.sum(self._last_pfo_counts[this_mask])
            curr_events_remaining = ak.sum(this_mask)
            return [
                curr_remaining,
                0.,
                100.,
                curr_remaining/curr_events_remaining]
        else:
            this_mask = self.get_curr_mask()
            self._last_pfo_counts = ak.count(this_mask, axis=1)
            curr_remaining = ak.sum(this_mask)
            curr_events_remaining = ak.num(this_mask, axis=0)
        return [curr_remaining,
                100 * curr_remaining/self._init_pfo_count,
                100 * curr_remaining/last_remaining,
                curr_remaining/curr_events_remaining]

    def _append_table_data(self, data):
        for i in range(len(self._pfo_table_cols)):
            self._table_data[self._pfo_table_cols[i]].append(data[i])
        return
    
    def _update_table_data(self, name):
        table_data = [name]
        table_data += self._get_remaining_events_data()
        table_data += self._get_remaining_pfos_data()
        self._append_table_data(table_data)
        self._concat_indicies[-1] += 1
        return

    def concatenate(self, other, return_copy=False):
        if not isinstance(other, CutHandler):
            raise NotImplementedError("CutHandler object can only be added to another CutHandler object")
        # Make sure the new set is consequtive
        if self._check_signature(other._signatures[0]) and (other._signatures[0][1] != -1):
            raise ValueError(f"Combining CutHandler objects requires consequtively matching signatures: "
                             + f"{self._end_sig} and {other._signatures[0]} do not match.")
        if return_copy:
            result = self.copy()
        else:
            result = self
        result._empty_curr_mask_queue()
        result._conseq_masks += other._conseq_masks
        result._curr_masks = other._curr_masks
        result._signatures += other._signatures[1:]
        result._start_sig = other._start_sig
        result.concat_index += 1
        result._concat_indicies += [i + len(result._concat_indicies) for i in other._concat_indicies]
        renormed_data = result._change_table_data_init_count(
            other,
            result._table_data["Remaining events"][0],
            result._init_pfo_count,
            last_pfos_count=result._last_pfo_counts)
        result._pfos_init |= other._pfos_init
        result._last_pfo_counts = other._last_pfo_counts
        for c in result._pfo_table_cols:
            result._table_data[c] += renormed_data[c][1:]
        if return_copy:
            return result
        else:
            return

    def __add__(self, other):
        return self.concatenate(other, return_copy=True)

    def _change_table_data_init_count(
            self, cut_obj,
            new_init_evts, new_init_pfos,
            last_events=None, last_pfos_count=None):
        new_data = cut_obj._table_data.copy()
        evt_counts = new_data["Remaining events"]
        new_evts_full = [100 * c/new_init_evts for c in evt_counts]
        new_evts_rel = new_data["Relative percentage events"]
        if last_events is not None:
            new_evts_rel[0] = 100 * evt_counts[0]/last_events
        pfo_counts = new_data["Remaining PFOs"]
        recreate_pfo_data = False
        if new_init_pfos != "-":
            if "-" not in pfo_counts:
                new_pfos_full = [100 * c/new_init_pfos for c in pfo_counts]
            else:
                # Need to add thing to catch case of second item being events based
                if last_pfos_count is not None:
                    recreate_pfo_data = True
                    counts_to_use = [np.sum(last_pfos_count[mask]) for mask in cut_obj._pre_pfo_init_masks]
                    counts_to_use += pfo_counts[1+len(counts_to_use):]
                    counts_to_use = [np.sum(last_pfos_count)] + counts_to_use
                    pfo_counts = counts_to_use
                    new_pfos_full = [100 * c/new_init_pfos for c in pfo_counts]
                else:
                    new_pfos_full = new_data["Percentage of total PFOs remaining"]
        else:
            new_pfos_full = new_data["Remaining PFOs"]
        if recreate_pfo_data:
            new_pfos_rel = [100.] + [100 * this/last for this, last in zip(counts_to_use[1:], counts_to_use[:-1])]
        else:
            new_pfos_rel = new_data["Relative percentage of PFOs"]
        if last_pfos_count is not None:
            new_pfos_rel[0] = 100 * pfo_counts/np.sum(last_pfos_count)
        if recreate_pfo_data:
            new_ave_pfos = [pfos/evts for pfos, evts in zip(evt_counts, pfo_counts)]
        else:
            new_ave_pfos = new_data["Average PFOs per event"]
        return {
            "Name":new_data["Name"],
            "Remaining events":evt_counts,
            "Percentage of total events remaining":new_evts_full,
            "Relative percentage events":new_evts_rel,
            "Remaining PFOs":pfo_counts,
            "Percentage of total PFOs remaining":new_pfos_full,
            "Relative percentage of PFOs":new_pfos_rel,
            "Average PFOs per event":new_ave_pfos}

    def _empty_curr_mask_queue(self):
        result = self._curr_masks[0]
        for m in self._curr_masks[1:]:
            result = result * m
        self._conseq_masks += [result]
        self._curr_masks = []
        return

    def add_mask(self, mask, name):
        # TODO need to decide if the mask needs to be stored separately:
        # This is needed if PFO is not set, and we are in an events-like (and simultaneous like, but probably ignore that for ease) signature
        this_sig = self._get_mask_signature(mask)
        if len(self._curr_masks) == 0 and len(self._conseq_masks) == 0:
            self._add_initial_data(mask)
        elif not self._check_signature(this_sig):
            self._empty_curr_mask_queue()
        self._curr_masks += [mask]
        # else:
            # If signature matches previous init signature, then add to self._curr_masks
            # Else if it matches the end signature, put the current mask into self._conseq_masks
            #   Reset current masks back to []
            # curr_sig = self._get_mask_signature(mask)
            # if len(self._curr_masks) == 1:
            #     # self._consequtive stores the initial state of
            #     # the first mask until set as a boolean here
            #     consequtive = curr_sig == self._consequtive
            #     if consequtive:
            #         self._signature = self._consequtive
            #     self._consequtive = consequtive
            # self._validate_signature(curr_sig)
            # if self._consequtive:
            #     self._signature = self._get_mask_signature(mask, end=True)
        self._signatures += [self._get_mask_signature(mask, end=True)]
        self._get_mask_signature(mask, update=True)
        self._update_table_data(name)
        return
        
    def get_curr_mask(self):
        if len(self._curr_masks) == 0:
            if len(self._conseq_masks) == 0:
                raise IndexError("No masks added")
            return self._conseq_masks[-1]
        result = self._curr_masks[0]
        for m in self._curr_masks[1:]:
            result = result * m
        return result
        # if len(self._curr_masks) == 0:
        #     raise IndexError("No masks added")
        # if self._consequtive:
        #     # N.B. also entered if only 1 mask is added
        #     # TODO this doesn't work! mask signature needs to be the initial signature still
        #     res = self._curr_masks[0]
        #     for mask in self._curr_masks[1:]:
        #         res = res[mask]
        # else:
        #     event_masks = []
        #     pfo_masks = []
        #     if not self._pfos_init:
        #         res = self._combine_masks(self._curr_masks)
        #     else:
        #         for i, sig in enumerate(self._signatures):
        #             if sig[1] == -1:
        #                 event_masks += [self._curr_masks[i]]
        #             else:
        #                 pfo_masks += [self._curr_masks[i]]
        #         event_mask = self._combine_masks(event_masks)
        #         res = self._combine_masks(pfo_masks)
        #         if event_mask is not None:
        #             res = event_mask * res
        # return res

    def apply_masks(self, data, return_table=False, application_index=0):
        result = data
        for m in self._conseq_masks[application_index:]:
            result = result[m]
        result = result[self.get_curr_mask()]
        if return_table:
            return result, self.get_table()
        else:
            return result

    def get_table(self, initial_concat_index=0, initial_name="Initial data", events_only=False):
        data = self._table_data
        if initial_concat_index > self.concat_index:
            raise IndexError(
                f"Concatenation index {initial_concat_index} out of range, "
                + f"only {self.concat_index} concatenation(s) present.")
        if initial_concat_index > 0:
            start_event_index = self._concat_indicies[initial_concat_index]
            data = self._change_table_data_init_count(
                self,
                data["Remaining events"][start_event_index],
                data["Remaining PFOs"][start_event_index])
            data["Name"][0] = initial_name
        if events_only:
            return pd.DataFrame({key:data[key] for key in self._event_table_cols})
        else:
            return pd.DataFrame(data)



In [4]:
size=1000
test_data = np.random.randint(0, 10, size=size)
curr_i = 0
data = []
while curr_i < size:
    this_size = np.random.randint(0,6)
    data += [test_data[curr_i:min(curr_i + this_size, size)]]
    curr_i += this_size
data = ak.Array(data)
print(data)
mask_1 = data > 2
print(mask_1)
mask_2 = data < 7
print(mask_2)
mask_3 = data >=4
print(mask_3)

[[1, 8, 9, 7, 9], [9, 6, 1], [3, 9], [4, 7, ... [2, 9, 9], [2], [4, 5, 1], [9, 4]]
[[False, True, True, True, True], [True, True, ... [True, True, False], [True, True]]
[[True, False, False, False, False], [False, ... [True, True, True], [False, True]]
[[False, True, True, True, True], [True, True, ... [True, True, False], [True, True]]


In [35]:
cutter_num = CutHandler()
cutter_num.get_curr_mask()

IndexError: No masks added

In [36]:
cutter_num.add_mask(mask_1, ">2")
print(cutter_num.get_curr_mask())
print("1 done")
cutter_num.add_mask(mask_2, "<7")
print(cutter_num.get_curr_mask())
print("2 done")
cutter_num.add_mask(mask_3, ">=4")
print(cutter_num.get_curr_mask())
print("3 done")
print(data[cutter_num.get_curr_mask()])
print(cutter_num._get_mask_signature(data))
print(cutter_num._get_mask_signature(data[cutter_num.get_curr_mask()]))

[[False, True, True, True, True], [True, True, ... [True, True, False], [True, True]]
1 done
[[False, False, False, False, False], [False, ... [True, True, False], [False, True]]
2 done
[[False, False, False, False, False], [False, ... [True, True, False], [False, True]]
3 done
[[], [6], [], [4], [6], [6], [], [], [5, ... [], [5, 6], [], [], [], [], [4, 5], [4]]
(429, 1000)
(429, 301)


In [37]:
print(cutter_num.apply_masks(data))
print(data[cutter_num.get_curr_mask()])
print(data)
cutter_num.get_table()

[[], [6], [], [4], [6], [6], [], [], [5, ... [], [5, 6], [], [], [], [], [4, 5], [4]]
[[], [6], [], [4], [6], [6], [], [], [5, ... [], [5, 6], [], [], [], [], [4, 5], [4]]
[[1, 8, 9, 7, 9], [9, 6, 1], [3, 9], [4, 7, ... [2, 9, 9], [2], [4, 5, 1], [9, 4]]


,Name,Remaining events,Percentage of total events remaining,Relative percentage events,Remaining PFOs,Percentage of total PFOs remaining,Relative percentage of PFOs,Average PFOs per event
0,Initial data,429,100.0,100.0,1000,100.0,100.000000,2.331002
1,>2,429,100.0,100.0,685,68.5,68.500000,1.596737
2,<7,429,100.0,100.0,394,39.4,57.518248,0.918415
3,>=4,429,100.0,100.0,301,30.1,76.395939,0.701632


In [8]:
cutter_num.get_table(events_only=True)

,Name,Remaining events,Percentage of total events remaining,Relative percentage events
0,Initial data,429,100.0,100.0
1,>2,429,100.0,100.0
2,<7,429,100.0,100.0
3,>=4,429,100.0,100.0


In [9]:
cutter_num.get_table(initial_concat_index=1)

IndexError: Concatenation index 1 out of range, only 0 concatenation(s) present.

In [18]:
dir(cutter_num)
['add_mask',
 'apply_masks',
 'concat_index',
 'concatenate',
 'copy',
 'get_curr_mask',
 'get_table']

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_initial_data',
 '_append_table_data',
 '_change_table_data_init_count',
 '_check_signature',
 '_concat_indicies',
 '_conseq_masks',
 '_curr_masks',
 '_empty_curr_mask_queue',
 '_end_sig',
 '_event_inds_indicies',
 '_event_table_cols',
 '_get_last_pfo_sig',
 '_get_mask_signature',
 '_get_remaining_events_data',
 '_get_remaining_pfos_data',
 '_init_pfo_count',
 '_last_pfo_counts',
 '_pfo_table_cols',
 '_pfos_init',
 '_signatures',
 '_start_sig',
 '_table_data',
 '_update_table_data',
 'add_mask',
 'apply_masks',
 'concat_index',
 'concatenate',
 'copy',
 'get_curr_mask',
 'get_table']

In [38]:
evts_cutter = CutHandler()
evt_1 = ak.num(data, axis=1) > 0
print(evt_1)
evt_2 = ak.num(data, axis=1) <= 3
print(evt_2)
evts_cutter.add_mask(evt_1, "No empty")
print(evts_cutter.get_curr_mask())
print("1 done")
evts_cutter.add_mask(evt_2, " At most 3 in event")
print(evts_cutter.get_curr_mask())
print("2 done")
print(data[evts_cutter.get_curr_mask()])
print(evts_cutter._get_mask_signature(data))
print(evts_cutter._get_mask_signature(evts_cutter.apply_masks(data)))
print(evts_cutter.apply_masks(data))
print(data[evts_cutter.get_curr_mask()])
print(data)
print(evts_cutter.get_table())
evts_cutter.get_table(events_only=True)

[True, True, True, True, True, True, False, ... True, True, True, True, True, True]
[False, True, True, True, True, False, True, ... True, True, True, True, True, True]
[True, True, True, True, True, True, False, ... True, True, True, True, True, True]
1 done
[False, True, True, True, True, False, False, ... True, True, True, True, True, True]
2 done
[[9, 6, 1], [3, 9], [4, 7, 7], [6], [5], ... 8], [2, 9, 9], [2], [4, 5, 1], [9, 4]]
(429, 1000)
(228, 450)
[[9, 6, 1], [3, 9], [4, 7, 7], [6], [5], ... 8], [2, 9, 9], [2], [4, 5, 1], [9, 4]]
[[9, 6, 1], [3, 9], [4, 7, 7], [6], [5], ... 8], [2, 9, 9], [2], [4, 5, 1], [9, 4]]
[[1, 8, 9, 7, 9], [9, 6, 1], [3, 9], [4, 7, ... [2, 9, 9], [2], [4, 5, 1], [9, 4]]
                  Name  Remaining events  \
0         Initial data               429   
1             No empty               351   
2   At most 3 in event               228   

   Percentage of total events remaining  Relative percentage events  \
0                            100.000000  

,Name,Remaining events,Percentage of total events remaining,Relative percentage events
0,Initial data,429,100.000000,100.000000
1,No empty,351,81.818182,81.818182
2,At most 3 in event,228,53.146853,64.957265


In [39]:
evts_cutter.get_table(events_only=True)

,Name,Remaining events,Percentage of total events remaining,Relative percentage events
0,Initial data,429,100.000000,100.000000
1,No empty,351,81.818182,81.818182
2,At most 3 in event,228,53.146853,64.957265


In [40]:
cutter_num.get_table()

,Name,Remaining events,Percentage of total events remaining,Relative percentage events,Remaining PFOs,Percentage of total PFOs remaining,Relative percentage of PFOs,Average PFOs per event
0,Initial data,429,100.0,100.0,1000,100.0,100.000000,2.331002
1,>2,429,100.0,100.0,685,68.5,68.500000,1.596737
2,<7,429,100.0,100.0,394,39.4,57.518248,0.918415
3,>=4,429,100.0,100.0,301,30.1,76.395939,0.701632


In [46]:
comb_cuts = cutter_num + evts_cutter

['Initial data', '>2', '<7', '>=4', 'No empty', ' At most 3 in event']
[429, 429, 429, 429, 351, 228]
[100.0, 100.0, 100.0, 100.0, 81.81818181818181, 53.14685314685315]
[100.0, 100.0, 100.0, 100.0, 81.81818181818181, 64.95726495726495]
[1000, 685, 394, 301, 1000, 450]
[100.0, 68.5, 39.4, 30.1, 100.0, 45.0]
[100.0, 68.5, 57.518248175182485, 76.39593908629442, 100.0, 45.0]
[2.331002331002331, 1.5967365967365967, 0.9184149184149184, 0.7016317016317016, 0.351, 0.5066666666666667]


In [41]:
print(type(cutter_num.copy()))
print(isinstance(cutter_num.copy(), CutHandler))
print(type(evts_cutter.copy()))
print(isinstance(evts_cutter.copy(), CutHandler))

<class '__main__.CutHandler'>
True
<class '__main__.CutHandler'>
True


In [42]:
x = cutter_num.copy()
print(isinstance(x, CutHandler))
x.concatenate(evts_cutter)

True
['Initial data', '>2', '<7', '>=4', 'No empty', ' At most 3 in event']
[429, 429, 429, 429, 351, 228]
[100.0, 100.0, 100.0, 100.0, 81.81818181818181, 53.14685314685315]
[100.0, 100.0, 100.0, 100.0, 81.81818181818181, 64.95726495726495]
[1000, 685, 394, 301, 1000, 450]
[100.0, 68.5, 39.4, 30.1, 100.0, 45.0]
[100.0, 68.5, 57.518248175182485, 76.39593908629442, 100.0, 45.0]
[2.331002331002331, 1.5967365967365967, 0.9184149184149184, 0.7016317016317016, 0.351, 0.5066666666666667]


In [44]:
x.get_table()

,Name,Remaining events,Percentage of total events remaining,Relative percentage events,Remaining PFOs,Percentage of total PFOs remaining,Relative percentage of PFOs,Average PFOs per event
0,Initial data,429,100.000000,100.000000,1000,100.0,100.000000,2.331002
1,>2,429,100.000000,100.000000,685,68.5,68.500000,1.596737
2,<7,429,100.000000,100.000000,394,39.4,57.518248,0.918415
3,>=4,429,100.000000,100.000000,301,30.1,76.395939,0.701632
4,No empty,351,81.818182,81.818182,1000,100.0,100.000000,0.351000
5,At most 3 in event,228,53.146853,64.957265,450,45.0,45.000000,0.506667


In [43]:
cutter_num.get_table()

,Name,Remaining events,Percentage of total events remaining,Relative percentage events,Remaining PFOs,Percentage of total PFOs remaining,Relative percentage of PFOs,Average PFOs per event
0,Initial data,429,100.0,100.0,1000,100.0,100.000000,2.331002
1,>2,429,100.0,100.0,685,68.5,68.500000,1.596737
2,<7,429,100.0,100.0,394,39.4,57.518248,0.918415
3,>=4,429,100.0,100.0,301,30.1,76.395939,0.701632


In [15]:
evts_cutter.get_table()

,Name,Remaining events,Percentage of total events remaining,Relative percentage events,Remaining PFOs,Percentage of total PFOs remaining,Relative percentage of PFOs,Average PFOs per event
0,Initial data,429,100.000000,100.000000,Not supplied,-,-,-
1,No empty,351,81.818182,81.818182,-,-,-,-
2,At most 3 in event,228,53.146853,64.957265,-,-,-,-


In [14]:
x.get_table()

,Name,Remaining events,Percentage of total events remaining,Relative percentage events,Remaining PFOs,Percentage of total PFOs remaining,Relative percentage of PFOs,Average PFOs per event
0,Initial data,429,100.0,100.0,1000,100.0,100.000000,2.331002
1,>2,429,100.0,100.0,685,68.5,68.500000,1.596737
2,<7,429,100.0,100.0,394,39.4,57.518248,0.918415
3,>=4,429,100.0,100.0,301,30.1,76.395939,0.701632


In [56]:
cutter2 = CutHandler()
small_data = data[evt_1][evt_2]
print(small_data)
smask_1 = small_data > 2
print(smask_1)
smask_2 = small_data < 7
print(smask_2)
smask_3 = small_data >=4
print(smask_3)
evts_cutter.add_mask(evt_1, "No empty")
print(evts_cutter.get_curr_mask())
print("1 done")
evts_cutter.add_mask(evt_2, " At most 3 in event")
print(evts_cutter.get_curr_mask())
print("2 done")
print(data[evts_cutter.get_curr_mask()])
print(evts_cutter._get_mask_signature(data))
print(evts_cutter._get_mask_signature(evts_cutter.apply_masks(data)))
print(evts_cutter.apply_masks(data))
print(data[evts_cutter.get_curr_mask()])
print(data)
print(evts_cutter.get_table())
evts_cutter.get_table(events_only=True)

(317, 688)

In [48]:
print(type(comb_cuts))
comb_cuts.get_table()

<class 'NotImplementedError'>


AttributeError: 'NotImplementedError' object has no attribute 'get_table'

In [69]:
print(cutter_num._get_mask_signature(mask_1))
print(cutter_num._get_mask_signature(mask_1, end=True))
print(cutter_num._get_mask_signature(mask_2))
print(cutter_num._get_mask_signature(mask_2, end=True))
print(cutter_num._get_mask_signature(mask_3))
print(cutter_num._get_mask_signature(mask_3, end=True))

(16, 30)
(11, 19)
(16, 30)
(13, 21)
(16, 30)
(9, 14)


In [25]:
(ak.sum(ak.any(mask_1, axis=1)), ak.sum(mask_1))

(25, 25)

In [29]:
ak.any(mask_1, axis=1)

<Array [[True], [True, ... [False, True]] type='16 * var * bool'>

In [30]:
mask_1

<Array [[[True]], ... [[False, True]]] type='16 * var * var * bool'>